# 리눅스 부트 프로세스

<img src="https://www.baeldung.com/wp-content/uploads/sites/2/2022/10/boot-process-662x1024.png" height="500" width="350">

- **BIOS (Basic Input/Output System)** : 운영 체제 중 가장 기본적인 소프트웨어이자 컴퓨터의 입출력을 처리하는 펌웨어
- **UEFI (Unified Extensible Firmware Interface)** : 통일 확장 펌웨어 인터페이스 또는 통합 확장 펌웨어 인터페이스는 운영 체제와 플랫폼 펌웨어 사이의 소프트웨어 인터페이스를 정의하는 규격이다.
- **Firmware** : 하드웨어를 low-level 컨트롤하기 위한 소프트웨어
- **Boot Loader** : a small program that loads the operating system
    - 운영체제가 시작되기 위해 필요한

- Kernel (핵심) : 리눅스의 핵심
    - 커널이 하드웨어 컨트롤 한다.

> 기술 질문) 부팅프로세스에 대해서 설명하시오
- Power on & BIOS 구동 -> MBR -> Boot Loader -> kernel -> 파일 시스템 마운트 -> 사용자 프로세스 실행

### Run levelss
- 싱글모드로 할 것인지, 멀티유저로 진행할지 확인하고 , level 선택\
: OS가 부팅이 된 이후, 어떤 상태로 시스템을 구동시킬지 결정하는 것

|Run levels|Target unit|설명|
|----|----|----|
|0|Poweroff.targer|시스템을 셧다운하고 종료|
|1|rescue.targer|복구 쉘 - 문제를 해결할 수 있는 용도로 사용/단일 사용자 환경(싱글모드)|
|2,3,4|multi-user.target|커맨드 환경의 다중사용자 모드|
|5|graphical.target|그래픽 환경의 다중사용자 모드(기본적으로 패키지를 설치해야 사용 가능)|
|6|reboot.target|시스템을 셧다운하고 재부팅|

`who -r` : 현재 런레벨 확인 가능한 명령어

### BIOS/UEFI를 통해 POST -> Boot Loader : kernel의 목록을 화면에 출력 -> kernel이 init process를 실행 -> init process가 run level을 결정 -> login

- Default Target
    - `system get-default` : 현재 default target 확인
    - `system set-default {target}` : {target}으로 default target 변경

# 복구모드
1. 루트 비밀번호 복구
2. fstab 등 잘못 설정해서 복구

### 루트 비밀번호 복구
1. 부팅하면서 e를 눌러 진입
2. linux16 문장에서 `UTF-8` 뒤에 `rd.break` 작성 후 `ctrl + x`로 나오기
    - 주의사항 : selinux가 활성화 되어 있다면 첫번째 rd 바로앞에 `selinux=0`를 추가하여 selinux를 끈다.
3. `rd.break`로 진입확인
4. `mount | grep -w '/sysroot'` : 마운트 진행
5. `mount -o remount,rw /sysroot` : 마운트 재진행
6. `chroot /sysroot` : chroot 명령을 사용하여 루트 디렉토리를 변경합니다. (중요) 루트 디렉토리를 변경하게 되면 파일에 대한 레이블정보가 전부 제거됩니다. 따라서 파일에 대한 레이블을 재지정해야 합니다.
7. `passwd` : 루트 비밀번호 변경
8. `touch /.autorelabel` : 레이블 재지정
9. `exit` : chroot에서 나오기
10. `exit` : rd.break 종료
11. 복구 확인하기

### 복구모드
- 부팅 진행중 마운트가 잘못되어 있다면 에러복구모드로 진입하게 됩니다.
- 일반적으로 fstab을 잘못건드려서 에러나는 케이스가 많습니다.
    - fstab 내용과 UUID 비교해보기


# 소프트웨어 패키지
> 특정 서비스를 운영하기 위해서 필요로 하는 프로그램 또는 도구를 쉽게 설치하고 관리할 수 있도록 하나의 패키지로 묶어서 제공하는 것

RPM(Redhat Package Manager)패키지 => `rpm`

`rpm -qa | grep httpd` : 패키지 확인하기

`httpd - 2.4.6 - 40.el7.centos . x86_64 . rpm`

- `httpd` : 패키지이름 - 설치 및 동작에 사용하는 명칭입니다.
- `2.4.6` : 버전정보 - 프로그램이 만들어진 버전 정보로 높을수록 최근에 만들어진 버전입니다.
- `40.el7.centos` : 릴리즈정보 - 패키지를 사용할 수 있는 리눅스 배포판의 버전이 명시됩니다.
- `x86_64` : 아키텍처 정보 - 지원하는 CPU 아키텍처의 정보입니다.
- `rpm` : 파일확장자 - RPM 패키지의 파일 확장자는 `.rpm`입니다.

`YUM`을 사용하여 패키지 관리\
: `RPM`기반의 패키지 설치, 제거, 업데이트를 관리해주는 도구

> `YUM`을 사용하기 위해서는 네트워크 연결이 필수\
: `YUP`(Yellowdog updater)를 대체\
: 패키지의 종속성을 알아서 해결해 주기 때문에 사용자 입장에서는 더 쉽게 설치하는 것이 가능

### `YUM` 저장소(리포지토리, repository)
> 패키지들을 저장해 놓은 하나의 서버를 의미
- repo(repository) : YUM 저장소에 연결할 수 있도록 설정한 파일

- 패키지 확인 명령어
    - `yum info {패키지}` : 패키지의 상세 정보
    - `yum provides {file name}` : file과 관련된 패키지 확인
    - `yum search {패키지}` : 패키지와 관련된 키워드를 사용하여 검색
    - `yum list` : 패키지 목록 확인

- 패키지 설치 명령어
    - `yum [-y] install {패키지 이름}` : 패키지 설치
    - `yum [-y] update {패키지 이름}` : 패키지 업데이트

- 패키지 삭제 명령어
    - `systemctl status {패키지}` : 삭제할 패키지가 실행중인지 확인
    - `systemctl stop {패키지}` : 패키지 중단
    - `yum remove {패키지}` : 패키지 제거

# 네트워크 관리

- `ifconfig` : ip주소 확인
- `ip a` : ip 확인
    - `<__,__,UP,__>` : UP이라는 단어가 있다면 네트워크 사용가능 활성화 가능상태다. UP인데 IP주소가 부여되지 않았다면 활성화를 안한 상태라고 보면 된다.
    - `ip {특정 인터페이스}` : 특정 인터페이스만 확인도 가능하다. a로 검색한 것과 출력결과가 다를 수 있다.

`nmtui` : NetworkManager Text User Interface는 텍스트 상자 형식의 설정 도구를 사용하여 터미널에서 사용자가 쉽게 네트워크 설정을 하도록 도와주는 도구입니다.

`tracerout {사이트}` : 연결된 라우트가 몇개인지 시각적으로 보여주는 것